In [ ]:
# Télécharger les ressources necéssaires pour utiliser le script
# !pip install python-binance
# !pip install pytz

import pip, os, pytz, pandas as pd
from datetime import date, timedelta, datetime
from market_imbalance import MarketImbalance, MarketCandle, create_market_candle, find_imbalances_after_fall, find_imbalances_after_rise, get_fullfilled_imbalances_after_fall, get_fullfilled_imbalances_after_rise, add_one_minute
from interact_with_binance import Trade, fetch_ohlcv_as_df, fetch_ohlcv, adjust_timestamps_to_local
from binance.client import Client

import nest_asyncio
import asyncio
import datetime
nest_asyncio.apply()
from binance import AsyncClient, BinanceSocketManager



In [ ]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket('BTCUSDT')
    last_candle_end = 1712484359
    async with ts as tscm:
        
        trades = []
        while True:
            res = await tscm.recv()
            trade = Trade(res)    
            # trades.append(Trade(res))
            # for trade in trades :
                # print(vars(trade))
            
    await client.close_connection()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [ ]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)

# Récupère les données de marché en *temps différé* sur Binance
interval_value = Client.KLINE_INTERVAL_1MINUTE
historical_candles_data = fetch_ohlcv(client, symbol = "ATOMUSDT", interval=interval_value, from_date= "31 mars 2024") #"1 Jan, 2015"

# Stocke les informations de chaque bougie dans un objet afin de pouvoir les retravailler plus facilement par la suite
historical_candles_object = [MarketCandle(*candle) for candle in historical_candles_data]

async def main(historical_candles_object, interval):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket('BTCUSDT') # start any sockets here, i.e a trade socket
    async with ts as tscm: # then start receiving messages
        
        # Permet d'assembler les trades en une nouvelle bougie en utilisant la fin de la dernière bougie comme borne
        last_candle = historical_candles_object[-1]
        last_candle_start = last_candle.timestamp
        last_candle_end = last_candle.close_time
        interval = datetime.timedelta(milliseconds=last_candle_end - last_candle_start)
        
        if trade.event_time > last_candle_end : 
            new_candle = MarketCandle()
            new_candle_start = trade.event_time
            new_candle_end = new_candle_start + interval
            print(new_candle_start, new_candle_end)
            
        elif trade.event_time ==  
        
        
        # updated_timestamp = add_one_minute(last_candle_timestamp)
        print(last_candle_timestamp, updated_timestamp)

        while trade.event_time <= updated_timestamp :
            new_candle = MarketCandle()
            new_candle.append(trade) # remplacer cette ligne par le code permettant d'obtenir les valeurs à stocker dans la classe
            
        else :
            historical_candles_object.append(new_candle)
        
        trades = []
        while True:
            res = await tscm.recv()

            
            trades.append(Trade(res))
            for trade in trades :
                print(vars(trade))
            
    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

In [ ]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)
#server_time = pd.to_datetime(client.get_server_time()['serverTime'], unit='ms')
#my_time = datetime.now(pytz.timezone('Europe/Paris'))

# Récupérer l'historique des prix pour une paire donnée depuis son listing sur la plateforme
# df = fetch_ohlcv_as_df(client, symbol = "BTCUSDT", interval=Client.KLINE_INTERVAL_1MINUTE, from_date= "31 mars 2024") #"1 Jan, 2015"
# df = adjust_timestamps_to_local(df, 'timestamp')

Les scripts ci-dessous permettent de vérifier les imbalances trouvés en les comparants avec les données de tradingview

In [ ]:
imbalances_after_fall = find_imbalances_after_fall(df)
fullfilled_imbalances_after_fall = get_fullfilled_imbalances_after_fall(imbalances_after_fall, df)
# for imb in fullfilled_imbalances_after_fall[-10:] : 
#    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

In [ ]:
imbalances_after_rise = find_imbalances_after_rise(df)
fullfilled_imbalances_after_rise = get_fullfilled_imbalances_after_rise(imbalances_after_rise, df)
# for imb in fullfilled_imbalances_after_rise[-10:] : 
#    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

Les scripts ci-dessous traitent de l'analyse des imbalances comblés

In [ ]:
# Percentage of gap filled after fall
percentage_filled_after_fall = (len(fullfilled_imbalances_after_fall) / len(imbalances_after_fall)) * 100
print(f'% of filled gap after fall = {percentage_filled_after_fall} with {len(fullfilled_imbalances_after_fall)} fullfilled imbalances on {len(imbalances_after_fall)} in total')

# Percentage of gap filled after rise
percentage_filled_after_rise = (len(fullfilled_imbalances_after_rise) / len(imbalances_after_rise)) * 100
print(f'% of filled gap after rise = {percentage_filled_after_rise} with {len(fullfilled_imbalances_after_rise)} fullfilled imbalances on {len(imbalances_after_rise)} in total')

Les scripts ci-dessous traitent de l'identification des imbalances non comblés, sous entendu des trades à prendre.
Notre objectif ici est :
1. Récupérer la liste des imbalances non comblés après une baisse
2. Récupérer la liste des imbalances non comblés après une hausse
3. Calculer la somme des imbalances incomblés
4. si le prix du marché pénètre une zone imbalance, alors ouvrir un trade pour l'imbalance concerné.
5. si le prix du marché pénètre la zone et déclenche notre stop-loss, alors rouvrir un trade.
6. si le prix du marché pénètre la zone et ressort de la zone sans déclencher notre stop-loss, alors conserver uniquement le trade déjà ouvert.

Notre stratégie n'est pas d'ouvrir un trade par imbalance non comblé. Notre stratégie est d'ouvrir un trade à chaque fois que le prix du marché pénètre une zone d'imbalance. Si le prix rentre dans la zone et déclenche notre stop-loss, nous devons réouvrir un trade lorsque le prix re-rentre dans la zone d'imbalance.


In [ ]:
# Fonctions pour identifier les imbalances non comblés après une baisse et après une hausse
unfilled_imbalance_after_fall = []
unfilled_imbalance_after_rise = []

for imb in imbalances_after_fall :
    if not imb.is_full_filled :
        unfilled_imbalance_after_fall.append(imb)
        
for imb in imbalances_after_rise :
    if not imb.is_full_filled :
        unfilled_imbalance_after_rise.append(imb)
        
# Affiche l'emsemble des imbalances
all_unfilled_imbalances = []
all_unfilled_imbalances.extend(unfilled_imbalance_after_fall)
all_unfilled_imbalances.extend(unfilled_imbalance_after_rise)


In [ ]:
for imb in unfilled_imbalance_after_fall[-10:] : 
    print('imbalance that started on', imb.timestamp, 'was full filled on', imb.was_fullfilled_at, 'within', imb.time_to_be_fullfilled, 'open price =', imb.open_price, 'close_price =', imb.close_price, 'delta =', round(imb.delta_to_be_filled_in, 2))

Les scripts ci dessous ont pour but de récupérer le prix actuel du marché

In [ ]:
import nest_asyncio
import asyncio
import datetime
nest_asyncio.apply()

from binance import AsyncClient, BinanceSocketManager


async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.trade_socket('BTCUSDT')
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            
            trade = Trade(res)
            print(trade.event_time, trade.event_type, trade.symbol, trade.price, trade.quantity)
            
    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Je reçois le dernier trade.
Quelles sont les informations que je reçois pour le trade ?
Quelles sont les informations que je reçois pour les bougies ?
Comment puis-je créer une bougie à partir des différents trades ?



In [ ]:
# Informations reçu pour les trades
{
  "e": "trade",     // Event type
  "E": 1672515782136,   // Event time
  "s": "BNBBTC",    // Symbol
  "t": 12345,       // Trade ID
  "p": "0.001",     // Price
  "q": "100",       // Quantity
  "b": 88,          // Buyer order ID
  "a": 50,          // Seller order ID
  "T": 1672515782136,   // Trade time
  "m": true,        // Is the buyer the market maker?
  "M": true         // Ignore
}

# Accéder aux différentes valeurs
            event_type = res["e"] # "trade"
            event_time = datetime.datetime.fromtimestamp(res['E'] / 1000, tz=datetime.timezone.utc) # 1672515782136
            event_time = event_time.astimezone(pytz.timezone('Europe/Paris'))
            symbol = res["s"] # "BNBBTC"
            trade_id = res["t"] # 12345
            price = res["p"] # "0.001"
            quantity = res["q"] # "100"
            buyer_order_id = res["b"] # 88
            selle_order_id = res["a"] # 50
            trade_time = datetime.datetime.fromtimestamp(res['T'] / 1000, tz=datetime.timezone.utc) # 1672515782136
            trade_time = trade_time.astimezone(pytz.timezone('Europe/Paris'))
            is_the_buyer_the_market_maker = res["m"] # true
            ignore = res["M"] # true

In [ ]:
# Informations reçu pour les bougies
timestamp=row['timestamp'],
open_price=row['open'],
high_price=row['high'],
low_price=row['low'],
close_price=row['close'],
volume=row['volume'],
close_time=row['close_time'],
quote_asset_volume=row['quote_asset_volume'],
number_of_trades=row['number_of_trades'],
taker_buy_base_asset_volume=row['taker_buy_base_asset_volume'],
taker_buy_quote_asset_volume=row['taker_buy_quote_asset_volume'],
ignore=row['ignore']

# Informations reçu pour les trades
{
  "e": "trade",     // Event type -> number_of_trades
  "E": 1672515782136,   // Event time -> timestamp
  "s": "BNBBTC",    // Symbol
  "t": 12345,       // Trade ID
  "p": "0.001",     // Price -> permet de définir open, high, low, close
  "q": "100",       // Quantity -> permet d'obtenir le volume
  "b": 88,          // Buyer order ID
  "a": 50,          // Seller order ID
  "T": 1672515782136,   // Trade time
  "m": true,        // Is the buyer the market maker?
  "M": true         // Ignore
}

In [ ]:
1) Récupérer les différents trades en provenance du websocket.
2) Voir si le prix ouvre ou ferme un trade
3) s'il ouvre un trade, envoyer le trade
4) sinon agréer les trades en une nouvelle bougie
5) ajouter la bougie à l'historique

on a un output du websocket.
on récupère le dernier trade et on l'ajoute dans une variable "nouvelle bougie"

le prix d'ouverture = le prix du trade
si le prix du nouveau trade > au prix d'ouverture & high = none :
    high = prix nouveau trade
si le prix du nouveau trade > au prix d'ouverture & > à high:
    high = prix nouveau trade
si le prix du nouveau trade < au prix d'ouverture & low = none :
    low = prix nouveau trade
si le prix du nouveau trade < au prix d'ouverture & < à low:
    low = prix nouveau trade
    
 while true :  
    trade = Trade(res)
    last_candle = MarketCandle()
    # manque la définition du timestamp de la nouvelle bougie
    last_candle.open_price = trade.price
    if trade.price >= last_candle.open_price and last_candle.high_price == None :
        last_candle.high_price = trade.price
    if trade.price >= last_candle.open_price and trade.price >= last_candle.high_price :
        last_candle.high_price = trade.price
    if trade.price <= last_candle.open_price and last_candle.low_price == None :
        last_candle.low_price = trade.price
    if trade.price <= last_candle.open_price and trade.price <= last_candle.low_price :
        last_candle.low_price = trade.price
    if trade.trade_time == historical_candles_object[-1].timestamp # comment dire si on attaque une nouvelle minute ?
    # comment dire si on attaque une nouvelle heure ?
    # comment dire si on attaque un nouveau quart d'heure ?
    
    for imbalance in unfilled_imbalance_after_fall :
        if last_candle.high_price >= imbalance.open_price :
            stop_loss = imbalance.open_price -((imbalance.close_price - imbalance.open_price)/2)
            print(f'buy 0.01 at market {imbalance.open_price} with a take profit at {imbalance.close_price} and a stop loss at {stop_loss}')
    


si le prix le plus haut de la nouvelle bougie >= à l'open_price alors ouvrir_un_long()
si le prix le plus haut de la nouvelle bougie >=
lorsqu'on commence une nouvelle bougie, la nouvelle ancienne bougie devient la derniere bougie et elle est ajouté à la liste des bougies.'

In [ ]:
'''# Importer la bibliothèque numpy pour les statistiques
import numpy as np

# Extraire les temps pour être rempli de la liste d'imbalances
time_to_be_filled = [imbalance.time_to_be_fullfilled for imbalance in fullfilled_imbalances_after_fall]
time_to_be_filled_seconds = [time.total_seconds() for time in time_to_be_filled]

# Calculer les statistiques
mean_time_to_fill = np.mean(time_to_be_filled)  # Moyenne
median_time_to_fill = np.median(time_to_be_filled)  # Médiane
min_time_to_fill = np.min(time_to_be_filled)  # Minimum
max_time_to_fill = np.max(time_to_be_filled)  # Maximum
std_time_to_fill = np.std(time_to_be_filled_seconds)  # Écart-type

# Calculer les percentiles
percentiles = np.percentile(time_to_be_filled, [25, 50, 75, 80, 85, 90, 95, 99])

# Afficher les statistiques
print("Mean Time to Fill: ", mean_time_to_fill)
print("Median Time to Fill: ", median_time_to_fill)
print("Min Time to Fill: ", min_time_to_fill)
print("Max Time to Fill: ", max_time_to_fill)
print("Standard Deviation of Time to Fill: ", std_time_to_fill)
print("25th Percentile: ", percentiles[0])
print("50th Percentile (Median): ", percentiles[1])
print("75th Percentile: ", percentiles[2])
print("80th Percentile: ", percentiles[3])
print("85th Percentile: ", percentiles[4])
print("90th Percentile: ", percentiles[5])
print("95th Percentile: ", percentiles[6])
print("99th Percentile: ", percentiles[7])'''

In [ ]:
'''# retourne une liste des jours où il y a eu des imbalances qui n'ont pas été comblés par la suite
random_imbalance_after_a_rise = find_imbalances_after_rise(df)

df_exp = pd.DataFrame(random_imbalance_after_a_rise, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df_exp['timestamp'] = pd.to_datetime(df_exp['timestamp'], unit='ms')
df_exp.set_index('timestamp', inplace=True)'''

In [ ]:
'''# code à modifier afin de pouvoir travailler avec tradingview sur des timeframes en heure et en minutes.
# Je suis limité à 5000 barres dans tradingview. Je dois donc réduire la liste des imbalances à ceux réalisés dans les 5000 derniers jours à partir d'aujourd'hui
# Get today's date
today = date.today()

# Calculate the date 5000 days ago
days_ago = today - timedelta(days=5000)
print(days_ago)

# Filter dates within the last 5000 days
recent_dates = [d for d in list_of_imbalance_start if d >= days_ago]

# Print recent dates
print("Recent dates within the last 5000 days:")
for d in recent_dates:
    print(d)'''